In [1]:
# Zum installieren von beatifulsoup benötigt
#! pip install beautifulsoup4



In [2]:
page = 'https://www.stepstone.de'

In [3]:

defaultLangueList =   ['Java', 'C#' , 'C++' , 'HTML','Python','C' ,
               'PHP', 'JavaScript', 'SQL', 'CSS']
numberOfSearch = 1




totalNumberOfAnnonces = 0 
conformAnnoces = 0
langueList =  []

for i in range (0,len(defaultLangueList)):
    langueList.append(defaultLangueList[i].lower())
pointList = createPointList()    



def createPointList():
    pointList =[]
    for item in langueList:
        pointList.append(0)
    return pointList

def createFoundList():
    foundList =[]
    for item in langueList:
        foundList.append(False)
    return foundList    


   

def mergeList(langue, point):
    if (len(langue) != len(point) ):
        print("Fehler: beide Listen sollten Gleichlang sein")
    for i in range (0,len(langue)):
        print (str(langue[i])+ ": " + str(point[i]))
        
        

In [4]:
from bs4 import BeautifulSoup
from datetime import datetime
import sys
import re
import requests
import lxml.html as lh
def getReport(url):
    global page ,pointList,defaultLangueList
    pointList = createPointList()
    urlList = getUrlFromWebsite(url,0)
    #print(urlList)
    print("starte Anzeigen Analyse")
    for link in urlList:
        text = getItemsFromURL(page+link)
       
        analyseText(text)
    doComprehension()
    
       
def getItemsFromURL(url):
        
        page = requests.post(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        week = []
        week = soup.find(class_ = 'js-app-ld-ContentBlock')
        if not str(week) :
            print ("gesuchte Klasse existiert nicht und muss ausgetauscht werden")
        text = ''
        for tag in week.find_all("li",recursive = True):
           
            s = ' '+ str(tag.text)+ ' '
            text = text + re.sub('[\d,.)(/-]', ' ', s)
        print (text)
        
        return text
    
class NoNextSite(Exception):
    """Tritt auf wenn die Seitensuche ihr maximum erreicht hat"""
    pass
    



def getUrlFromWebsite(url, numberSearched):
    global numberOfSearch
    page = requests.post(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    urlList = []
    for a in soup.find_all('a', href=True):
        if ('stellenangebote' in str(a) and a['href'] not in urlList):
            urlList.append (a['href'])
          
    newUrl = soup.find(direction = 'right')
   
    try: 
        testNextUrL = newUrl['href']
        if numberOfSearch-1 == numberSearched:
            raise NoNextSite
    except KeyError:
        print("finished url search")
    except NoNextSite:
        print("finished url search")
    else:
        
        urlList = urlList + getUrlFromWebsite(newUrl['href'],numberSearched +1)
    
       
   
    return urlList    
   
    
# funktion analysiert ob eine sprache in einem text genannt wird 
# in diesem fall wird davon ausgegangen dass genannt = vorraussetzung für den Beruf ist
def analyseText(text):
    global langueList , pointList, totalNumberOfAnnonces,conformAnnoces
    foundList = createFoundList()
    for word in str(text).split(' ') :
        if(word.lower() in langueList):
            for i in range (len(langueList)):
                if (langueList[i] == word.lower() and foundList[i] == False):
                    pointList[i] = pointList[i] +1 
                    foundList[i] = True         
    if(True in foundList):
        conformAnnoces = conformAnnoces +1
    totalNumberOfAnnonces = totalNumberOfAnnonces +1    
    
    
    
def doComprehension():
    global pointList, totalNumberOfAnnonces,conformAnnoces
    mergeList(defaultLangueList, pointList) 
    print("Bei " + str(totalNumberOfAnnonces) +" überprüften Anzeigen, wovon bei " + str(conformAnnoces) + 
          " mindestens eine dem Programm bekannte Sprachen dabei waren.")
    for i in range (len(pointList)):
        pointList[i] = str(pointList[i]/conformAnnoces)[:5]+ "%" 
    mergeList(defaultLangueList, pointList) 




In [5]:
url = 'https://www.stepstone.de/5/ergebnisliste.html?stf=freeText&ns=1&qs=%5B%7B%22id%22%3A%22216805%22%2C%22description%22%3A%22Software-Entwickler%2Fin%22%2C%22type%22%3A%22jd%22%7D%5D&companyID=0&cityID=0&sourceOfTheSearchField=resultlistpage%3Ageneral&searchOrigin=Resultlist_top-search&ke=Software-Entwickler%2Fin&ws=&ra=30'

#url = 'https://www.stepstone.de/5/ergebnisliste.html?stf=freeText&ns=1&companyid=0&sourceofthesearchfield=resultlistpage%3Ageneral&qs=[{"id"%3A216805%2C"description"%3A"Software-Entwickler\%2Fin"%2C"type"%3A"jd"}]&ke=Software-Entwickler%2Fin&ra=30&suid=c4f83374-3466-4871-be7a-c171dfa6cb57&li=100&of=0&action=per_page_changed'
getReport(url)

finished url search
starte Anzeigen Analyse
 Entwicklung von softwarebasierten Vernetzungslösungen  Zusammenarbeit mit externen Dienstleistern und Forschungsinstituten  Entwicklung von neuen Vernetzungslösungen und  technologien in einem interdisziplinären Team  Überführung der Technologieprojekte in die Serienentwicklung  Studium der Informatik  Fundierte Programmiererfahrungen von IoT  oder Cloud Lösungen  Umfangreiche Kenntnisse in den gängigen Programmiersprachen  Blockchain und Distributed Ledger Erfahrungen sind wünschenswert  Selbstständigkeit und Kommunikationsstärke im Umgang mit internen und externen Partnern  Sehr gute Englisch Kenntnisse 
 Anforderungsermittlung gemeinsam mit Anwendern  Konzeption  Entwicklung und Umsetzung softwaretechnischer Lösungen und Prototypen  inkl  Softwarearchitektur  in Zusammenarbeit mit Partnern  Evaluation verschiedener Technologien und Geschäftsmodelle und ggf  Anpassung an Anwendungsfälle im Forschungsprojekt  ggf  Publikation und Präsentati

 Agiles Arbeiten mit dem Scrum Framework und modernen Technologien  Das wahre Problem hinter den beschriebenen Anforderungen schon vor dem Programmieren erkennen  Beteiligung an Diskussionen über Lösungen und neue Ansätze  um die Entwicklunsgrichtung zu beeinflussen  Verantwortung für den Code übernehmen  Abgeschlossenes Hochschulstudium oder Berufsausbildung  bevorzugt im IT Bereich  Fundierte Kenntnisse in der agilen Softwareentwicklung  Gute Englischkenntnisse  um sich auf internationaler Ebene auszutauschen  Verständnis von Softwareentwicklung als ein Handwerk  das Talent und ständiges Lernen erfordert  Hohe Bereitschaft neues Wissen anzueignen und dieses anzuwenden  Klare und einfache Ausdrucksweise  Kostenfreie Bereitstellung von frischem Obst  Kaffee  Tee und Wasser  Massage sowie vergünstigte Mitgliedschaft im Fitnessstudio  Kostenfreie Parkmöglichkeiten vor Ort  Gemeinsame Mitarbeiterevents  Flexibles Arbeitszeitmodell in einem modernen Arbeitsumfeld  Fundierte Einarbeitung du

 Initiale Erstellung von PowerShell Skripten zur Automation der Installationsroutinen der Citrix Virtual Apps Infrastruktur  Automation von weiteren Arbeitsprozessen und Anwendungsinstallationen  Weiterentwicklung und Anpassung der Skripte an die jeweiligen Citrix Virtual Apps Versionen  Integration der Automationsskripte in die SRM AO Workflows  Qualitätssicherung mittels Erstellung und Durchführung von Integrations  und Systemtestfällen  Unterstützung der Kollegen bei der Ausbringung von Citrix Cloud Platform Releases  Erstellung und Pflege der Dokumentation  Abstimmungen von technischen Anforderungen mit dem Bereich IT Plattformen  Abgeschlossenes Studium der  Wirtschafts Informatik sowie Kenntnisse aus mehrjähriger Berufserfahrung  Sehr gute Kenntnisse im Umgang mit PowerShell und SCCM  Idealerweise Kenntnisse bzgl  System  und Citrixadministration unter Windows Server      R  Windows Server       Mehrjährige Erfahrung in der Erstellung von PowerShell Skripten  Eigenverantwortliche

 In unserem internationalen Entwicklungsteam arbeitest Du am Design und der Realisierung einer Cloud basierten IoT Lösung   Dabei beschäftigst Du Dich insbesondere mit dem Bereich der Kommunikationsdienste und Backends   Als Entwickler für Cloud Solutions gehören die Anforderungsanalyse  die Implementierung und die Dokumentation der Softwarearchitekturen zu Deinen Aufgaben   Auch Modul   Integrations  und Systemtests zu planen und umzusetzen  ist Teil Deiner Tätigkeiten   Studium der Informatik oder vergleichbar   Erste praktische Erfahrungen in der Software Entwicklung mit Linux  Node js und Datenbanken   Kenntnisse im Bereich von Cloud basierten Systemen und automatisierten Software Tests sind ein Plus aber kein Muss   Für Dich ist ein hohes Qualitätsbewusstsein bei Deiner täglichen Arbeit selbstverständlich   Im Team punktest Du mit Deiner Kommunikationsstärke sowohl auf Deutsch als auch auf Englisch   Eine abwechslungsreiche und anspruchsvolle Tätigkeit mit neuen Herausforderungen 

 Entwicklung von containerbasierten Anwendungen und Softwarelösungen u  a  auf Java Basis  Unterstützung bei der Erstellung von Konzepten  Architekturen und wiederverwendbarer Services  Anforderungsanalyse und Spezifikation von Lösungen im Umfeld der Anwendungsentwicklung in Zusammenarbeit mit Fachabteilungen  Auswahl von geeigneten Frameworks  Aufbau und Weiterentwicklung einer geeigneten Continuous Integration  bzw  Contiuous Delivery Pipeline  „Toolgestützte“ Sicherstellung der Softwarequalität durch Modul  und Unittests sowie Analyse von Fehlern  Mitgestaltung und Einführung von DevOps   Prozessen   Sie verfügen über ein abgeschlossenes Studium der Informatik  Wirtschaftsinformatik oder einen vergleichbaren Studiengang mit Schwerpunkt Informatik  Sie besitzen Kenntnisse über die Entwicklung von Microservices  eine Containerverwaltungsplattform und Microservices Architekturen  Sie sind vertraut mit Programmiersprachen wie Java  Python und Node js   Kommunikationsstärke  Teamfähigkei